In [52]:
from PIL import Image, ImageFont, ImageDraw 
import glob
import string
import random
import os
import shutil
import math
import tensorflow as tf
print(tf.__version__)
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, Rescaling
from tensorflow.keras import Input 
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
import cv2

width = 128
height = 128

2.9.0


In [53]:
data_path = "C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data"
ext = ["jpg","png", "jpeg"];

def read_img(path):
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (width, height))
    return img

In [54]:
imgs = []
labels = []

for path in glob.glob(f'{data_path + "/True"}/*/'):
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        for file in files:
            imgs.append(read_img(file))
            labels.append(1.0)

for path in glob.glob(f'{data_path + "/False"}/*/'):
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        for file in files:
            imgs.append(read_img(file))
            labels.append(0.0)
            
print("Imgs len: ", len(imgs))
print("Labels len: ", len(labels))

Imgs len:  510
Labels len:  510


In [55]:
imgs, labels = shuffle(imgs, labels)

In [56]:
imgs = np.array(imgs) / 256.0
labels = np.array(labels)

In [57]:
x_train, x_test, y_train, y_test = train_test_split(imgs, labels, train_size=0.8, random_state=1) 

In [58]:
print(x_train.shape)
print(x_test.shape)

(408, 128, 128, 3)
(102, 128, 128, 3)


In [59]:
model = keras.Sequential()

model.add(keras.Input(shape=(width, height, 3))) #3 if RGB, 1 if GrayScale

model.add(tf.keras.layers.Conv2D(64, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(64, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(32, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(32, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(16, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(Flatten())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(64, activation="relu"))

model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [60]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 128, 128, 64)      4864      
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 64, 64, 64)       0         
 g2D)                                                            
                                                                 
 batch_normalization_15 (Bat  (None, 64, 64, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_16 (Conv2D)          (None, 64, 64, 64)        102464    
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                            
                                                      

In [61]:
model.compile(optimizer=Adam(learning_rate=0.0005), loss="MeanSquaredError")

In [62]:
early_stop = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=20, epochs=100, callbacks=[early_stop])

Epoch 1/100
21/21 [==============================] - 3s 71ms/step - loss: 0.1101 - val_loss: 0.2453
Epoch 2/100
21/21 [==============================] - 1s 46ms/step - loss: 0.0635 - val_loss: 0.2081
Epoch 3/100
21/21 [==============================] - 1s 45ms/step - loss: 0.0693 - val_loss: 0.1940
Epoch 4/100
21/21 [==============================] - 1s 46ms/step - loss: 0.0559 - val_loss: 0.1792
Epoch 5/100
21/21 [==============================] - 1s 44ms/step - loss: 0.0542 - val_loss: 0.1905
Epoch 6/100
21/21 [==============================] - 1s 44ms/step - loss: 0.0518 - val_loss: 0.1847
Epoch 7/100
21/21 [==============================] - 1s 44ms/step - loss: 0.0637 - val_loss: 0.2534
Epoch 8/100
21/21 [==============================] - 1s 45ms/step - loss: 0.0572 - val_loss: 0.2594
Epoch 9/100
21/21 [==============================] - 1s 46ms/step - loss: 0.0518 - val_loss: 0.3191


In [63]:
model.save('Face_recognition.h5')

In [66]:
result = model.predict(x_test).flatten()
print(result)

4/4 [==============================] - 0s 81ms/step
[0.61261475 0.6080866  0.61300164 0.6013592  0.6125489  0.60969484
 0.6080013  0.6140954  0.6123718  0.61303085 0.608053   0.6105988
 0.60859513 0.60378456 0.6053471  0.6068632  0.61263824 0.6084306
 0.6115178  0.61287457 0.61559206 0.6133717  0.61373067 0.606746
 0.6056151  0.61411595 0.6121     0.60971636 0.6063653  0.6122075
 0.6084388  0.6114573  0.6124323  0.60968596 0.6078933  0.61166465
 0.60896456 0.611402   0.60669    0.6111899  0.61929107 0.6083842
 0.61107945 0.60959816 0.6116034  0.61129296 0.6039132  0.6014029
 0.614421   0.61325926 0.6114429  0.60952276 0.61415446 0.6127305
 0.6216098  0.60522723 0.60863465 0.6100095  0.6219999  0.61188245
 0.6116677  0.60754293 0.6094993  0.6113127  0.6073763  0.6053808
 0.61207235 0.61254764 0.6158874  0.6234371  0.6098403  0.6101206
 0.61129373 0.62037575 0.61172706 0.6117749  0.61252576 0.6067425
 0.60644364 0.61148    0.6028615  0.61346847 0.6066234  0.61971736
 0.6135782  0.6125244